In [9]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 147.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 369.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3


  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
Using cached transformers-4.57.3-py3-none-any.whl (12.0 MB)
Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [10]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
# logging.set_verbosity_error()

In [11]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import get_peft_model, LoraConfig, TaskType
from torch.utils.data import DataLoader
import numpy as np
print("torch:", torch.__version__)
print("cuda devices:", torch.cuda.device_count())

torch: 2.9.0+cu126
cuda devices: 1


In [12]:
import os
import warnings
import torch
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    logging
)
from peft import get_peft_model, LoraConfig
import evaluate

In [13]:
# Silence all unnecessary outputs
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
logging.set_verbosity_error()


In [14]:
dataset = load_dataset("glue", "sst2")

# 2️⃣ Model checkpoint and labels
model_checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative": 0, "Positive": 1}

# 3️⃣ Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint,
    num_labels=2,
    id2label=id2label,
    label2id=label2id,
    device_map="auto"
)

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [15]:
from transformers import AutoTokenizer, DataCollatorWithPadding

# 4️⃣ Load tokenizer and fix pad token
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

# TinyLlama / LLaMA-style tokenizer does NOT need add_prefix_space
# (that's mostly for GPT-2-like models)
# So we don't pass add_prefix_space=True here.

if tokenizer.pad_token is None:
    # For LLaMA-like models, use eos_token as pad_token
    tokenizer.pad_token = tokenizer.eos_token
    model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

MAX_LEN = 128  # SST-2 sentences are short; 128 is usually plenty


def clean_text(s):
    if s is None:
        return ""
    # strip whitespace + collapse multiple spaces/newlines
    return " ".join(str(s).strip().split())


def tokenize_function(examples):
    # 🧠 Make it more "instructional" for a causal LM
    texts = [
        f"Review: {clean_text(x)}\nSentiment:"
        for x in examples["sentence"]
    ]

    return tokenizer(
        texts,
        truncation=True,
        padding=False,   # ✅ dynamic padding via DataCollator
        max_length=MAX_LEN,
    )


# Remove raw text + idx, keep label
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["sentence", "idx"],
)

# Trainer expects "labels" by default
if "label" in tokenized_dataset["train"].column_names:
    tokenized_dataset = tokenized_dataset.rename_column("label", "labels")

tokenized_dataset.set_format("torch")

# 6️⃣ Data collator handles dynamic padding
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    pad_to_multiple_of=8,  # good for GPU tensor cores
)


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [16]:
accuracy = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    preds = np.argmax(preds, axis=1)
    return {"accuracy": accuracy.compute(predictions=preds, references=labels)}

# 8️⃣ LoRA config
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ]
)
model = get_peft_model(model, peft_config)

In [17]:
training_args = TrainingArguments(
    disable_tqdm=False,
    output_dir="./SST2_adaptive_results",
    num_train_epochs=6,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=2,
    logging_steps=500,
    eval_strategy="epoch",
    eval_steps=500,
    bf16=True,              
    report_to="none",
    save_strategy="no",
)


In [18]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [23]:

from torch.utils.data import DataLoader
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["validation"]

val_dataloader = DataLoader(
    eval_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=data_collator
)

In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [22]:
met = trainer.evaluate()
print(met)

{'eval_loss': 0.613085925579071, 'eval_accuracy': {'accuracy': 0.7}, 'eval_runtime': 1.0523, 'eval_samples_per_second': 9.503, 'eval_steps_per_second': 4.752, 'epoch': 6.0}
